In [1]:
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import pandas as pd
import json
import wandb
import os

/Users/kyle/miniconda3/envs/eval-aug/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
project_name = "healthyml/In-Context Domain Transfer Improves Out-of-Domain Robustness"
table_paths = {
    # BOSS Sentiment
    # ID
    # BERT
    "BOSS_Sentiment_ID_BERT_Insert": "run-omxsb0i7-validation_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_BERT_Substitute": "run-omxsb0i7-validation_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_BERT_Translate": "run-omxsb0i7-validation_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_BERT_Paraphrase": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_BERT_ICR": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_ID_T5_Insert": "run-omxsb0i7-validation_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Substitute": "run-omxsb0i7-validation_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Translate": "run-omxsb0i7-validation_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Paraphrase": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_ID_T5_ICR": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

    # Falcon
    "BOSS_Sentiment_ID_Falcon_Insert": "run-omxsb0i7-validation_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_ID_Falcon_Substitute": "run-omxsb0i7-validation_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_ID_Falcon_Translate": "run-omxsb0i7-validation_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_ID_Falcon_Paraphrase": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_ID_Falcon_ICR": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # SST5
    # BERT
    "BOSS_Sentiment_SST5_BERT_Insert": "run-omxsb0i7-sst5_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_Substitute": "run-omxsb0i7-sst5_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_Translate": "run-omxsb0i7-sst5_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_Paraphrase": "run-omxsb0i7-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_ICR": "run-omxsb0i7-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Insert": "run-gdqehris-sst5_aug_insert_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Substitute": "run-gdqehris-sst5_aug_substitute_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Translate": "run-gdqehris-sst5_aug_backtranslate_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Paraphrase": "run-gdqehris-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_ICR": "run-gdqehris-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Insert": "run-gfb8nevp-sst5_aug_insert_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Substitute": "run-gfb8nevp-sst5_aug_substitute_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Translate": "run-gfb8nevp-sst5_aug_backtranslate_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Paraphrase": "run-gfb8nevp-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT3000_ICR": "run-gfb8nevp-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Insert": "run-hg6u172t-sst5_aug_insert_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Substitute": "run-hg6u172t-sst5_aug_substitute_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Translate": "run-hg6u172t-sst5_aug_backtranslate_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Paraphrase": "run-hg6u172t-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT6000_ICR": "run-hg6u172t-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SST5_BERT12000_Insert": "run-pv6dxpzt-sst5_aug_insert_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT12000_Substitute": "run-pv6dxpzt-sst5_aug_substitute_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT12000_Translate": "run-pv6dxpzt-sst5_aug_backtranslate_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT12000_Paraphrase": "run-pv6dxpzt-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT12000_ICR": "run-pv6dxpzt-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SST5_BERT24000_Insert": "run-ujjjnolk-sst5_aug_insert_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT24000_Substitute": "run-ujjjnolk-sst5_aug_substitute_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT24000_Translate": "run-ujjjnolk-sst5_aug_backtranslate_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT24000_Paraphrase": "run-ujjjnolk-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT24000_ICR": "run-ujjjnolk-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_SST5_T5_Insert": "run-omxsb0i7-sst5_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_Substitute": "run-omxsb0i7-sst5_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_Translate": "run-omxsb0i7-sst5_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_Paraphrase": "run-omxsb0i7-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_ICR": "run-omxsb0i7-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

    # Falcon
    "BOSS_Sentiment_SST5_Falcon_Insert": "run-omxsb0i7-sst5_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SST5_Falcon_Substitute": "run-omxsb0i7-sst5_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SST5_Falcon_Translate": "run-omxsb0i7-sst5_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SST5_Falcon_Paraphrase": "run-omxsb0i7-sst5_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SST5_Falcon_ICR": "run-omxsb0i7-sst5_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # SemEval
    # BERT
    "BOSS_Sentiment_SemEval_BERT_Insert": "run-omxsb0i7-semval_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_Substitute": "run-omxsb0i7-semval_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_Translate": "run-omxsb0i7-semval_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_Paraphrase": "run-omxsb0i7-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_ICR": "run-omxsb0i7-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SemEval_BERT1500_Insert": "run-gdqehris-semval_aug_insert_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT1500_Substitute": "run-gdqehris-semval_aug_substitute_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT1500_Translate": "run-gdqehris-semval_aug_backtranslate_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT1500_Paraphrase": "run-gdqehris-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT1500_ICR": "run-gdqehris-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SemEval_BERT3000_Insert": "run-gfb8nevp-semval_aug_insert_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT3000_Substitute": "run-gfb8nevp-semval_aug_substitute_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT3000_Translate": "run-gfb8nevp-semval_aug_backtranslate_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT3000_Paraphrase": "run-gfb8nevp-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT3000_ICR": "run-gfb8nevp-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SemEval_BERT6000_Insert": "run-hg6u172t-semval_aug_insert_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT6000_Substitute": "run-hg6u172t-semval_aug_substitute_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT6000_Translate": "run-hg6u172t-semval_aug_backtranslate_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT6000_Paraphrase": "run-hg6u172t-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT6000_ICR": "run-hg6u172t-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SemEval_BERT12000_Insert": "run-pv6dxpzt-semval_aug_insert_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT12000_Substitute": "run-pv6dxpzt-semval_aug_substitute_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT12000_Translate": "run-pv6dxpzt-semval_aug_backtranslate_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT12000_Paraphrase": "run-pv6dxpzt-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT12000_ICR": "run-pv6dxpzt-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SemEval_BERT24000_Insert": "run-ujjjnolk-semval_aug_insert_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT24000_Substitute": "run-ujjjnolk-semval_aug_substitute_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT24000_Translate": "run-ujjjnolk-semval_aug_backtranslate_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT24000_Paraphrase": "run-ujjjnolk-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT24000_ICR": "run-ujjjnolk-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_SemEval_T5_Insert": "run-omxsb0i7-semval_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_Substitute": "run-omxsb0i7-semval_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_Translate": "run-omxsb0i7-semval_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_Paraphrase": "run-omxsb0i7-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_ICR": "run-omxsb0i7-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

    # Falcon
    "BOSS_Sentiment_SemEval_Falcon_Insert": "run-omxsb0i7-semval_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SemEval_Falcon_Substitute": "run-omxsb0i7-semval_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SemEval_Falcon_Translate": "run-omxsb0i7-semval_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SemEval_Falcon_Paraphrase": "run-omxsb0i7-semval_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SemEval_Falcon_ICR": "run-omxsb0i7-semval_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # Dynasent
    # BERT
    "BOSS_Sentiment_Dynasent_BERT_Insert": "run-omxsb0i7-dynasent_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_Substitute": "run-omxsb0i7-dynasent_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_Translate": "run-omxsb0i7-dynasent_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_Paraphrase": "run-omxsb0i7-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_ICR": "run-omxsb0i7-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT1500_Insert": "run-gdqehris-dynasent_aug_insert_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT1500_Substitute": "run-gdqehris-dynasent_aug_substitute_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT1500_Translate": "run-gdqehris-dynasent_aug_backtranslate_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT1500_Paraphrase": "run-gdqehris-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT1500_ICR": "run-gdqehris-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT3000_Insert": "run-gfb8nevp-dynasent_aug_insert_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT3000_Substitute": "run-gfb8nevp-dynasent_aug_substitute_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT3000_Translate": "run-gfb8nevp-dynasent_aug_backtranslate_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT3000_Paraphrase": "run-gfb8nevp-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT3000_ICR": "run-gfb8nevp-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT6000_Insert": "run-hg6u172t-dynasent_aug_insert_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT6000_Substitute": "run-hg6u172t-dynasent_aug_substitute_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT6000_Translate": "run-hg6u172t-dynasent_aug_backtranslate_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT6000_Paraphrase": "run-hg6u172t-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT6000_ICR": "run-hg6u172t-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT12000_Insert": "run-pv6dxpzt-dynasent_aug_insert_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT12000_Substitute": "run-pv6dxpzt-dynasent_aug_substitute_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT12000_Translate": "run-pv6dxpzt-dynasent_aug_backtranslate_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT12000_Paraphrase": "run-pv6dxpzt-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT12000_ICR": "run-pv6dxpzt-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT24000_Insert": "run-ujjjnolk-dynasent_aug_insert_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT24000_Substitute": "run-ujjjnolk-dynasent_aug_substitute_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT24000_Translate": "run-ujjjnolk-dynasent_aug_backtranslate_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT24000_Paraphrase": "run-ujjjnolk-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT24000_ICR": "run-ujjjnolk-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_Dynasent_T5_Insert": "run-omxsb0i7-dynasent_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_Substitute": "run-omxsb0i7-dynasent_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_Translate": "run-omxsb0i7-dynasent_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_Paraphrase": "run-omxsb0i7-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_ICR": "run-omxsb0i7-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

    # Falcon
    "BOSS_Sentiment_Dynasent_Falcon_Insert": "run-omxsb0i7-dynasent_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_Dynasent_Falcon_Substitute": "run-omxsb0i7-dynasent_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_Dynasent_Falcon_Translate": "run-omxsb0i7-dynasent_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_Dynasent_Falcon_Paraphrase": "run-omxsb0i7-dynasent_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_Dynasent_Falcon_ICR": "run-omxsb0i7-dynasent_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # BOSS Toxicity
    # ID
    # BERT
    "BOSS_Toxicity_ID_BERT_Insert": "run-57a9z4m8-validation_aug_insert_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ID_BERT_Substitute": "run-57a9z4m8-validation_aug_substitute_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ID_BERT_Translate": "run-57a9z4m8-validation_aug_backtranslate_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ID_BERT_Paraphrase": "run-cfj9o5h7-validation_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ID_BERT_ICR": "run-dvdcuuij-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Toxicity_ID_T5_Insert": "run-0ctkgl69-validation_aug_insert_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ID_T5_Substitute": "run-0ctkgl69-validation_aug_substitute_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ID_T5_Translate": "run-0ctkgl69-validation_aug_backtranslate_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ID_T5_Paraphrase": "run-0ctkgl69-validation_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ID_T5_ICR": "run-0ctkgl69-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",

    # Falcon
    "BOSS_Toxicity_ID_Falcon_Insert": "run-7nqc8p48-validation_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ID_Falcon_Substitute": "run-mzhz5px0-validation_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ID_Falcon_Translate": "run-gx7ucgnh-validation_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ID_Falcon_Paraphrase": "run-lock4bx1-validation_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ID_Falcon_ICR": "run-tr8mluz2-validation_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # Toxigen
    # BERT
    "BOSS_Toxicity_Toxigen_BERT_Insert": "run-57a9z4m8-toxigen_aug_insert_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Toxigen_BERT_Substitute": "run-57a9z4m8-toxigen_aug_substitute_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Toxigen_BERT_Translate": "run-57a9z4m8-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Toxigen_BERT_Paraphrase": "run-57a9z4m8-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Toxigen_BERT_ICR": "run-57a9z4m8-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT3000_Insert": "run-2ecsgmy9-toxigen_aug_insert_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT3000_Substitute": "run-2ecsgmy9-toxigen_aug_substitute_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT3000_Translate": "run-2ecsgmy9-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT3000_Paraphrase": "run-2ecsgmy9-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT3000_ICR": "run-2ecsgmy9-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT6000_Insert": "run-ms2943fk-toxigen_aug_insert_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT6000_Substitute": "run-ms2943fk-toxigen_aug_substitute_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT6000_Translate": "run-ms2943fk-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT6000_Paraphrase": "run-ms2943fk-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT6000_ICR": "run-ms2943fk-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT12000_Insert": "run-7hgtyr98-toxigen_aug_insert_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT12000_Substitute": "run-7hgtyr98-toxigen_aug_substitute_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT12000_Translate": "run-7hgtyr98-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT12000_Paraphrase": "run-7hgtyr98-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT12000_ICR": "run-7hgtyr98-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT24000_Insert": "run-wl2a38kf-toxigen_aug_insert_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT24000_Substitute": "run-wl2a38kf-toxigen_aug_substitute_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT24000_Translate": "run-wl2a38kf-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT24000_Paraphrase": "run-wl2a38kf-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT24000_ICR": "run-wl2a38kf-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT48000_Insert": "run-iaorx9o7-toxigen_aug_insert_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT48000_Substitute": "run-iaorx9o7-toxigen_aug_substitute_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT48000_Translate": "run-iaorx9o7-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT48000_Paraphrase": "run-iaorx9o7-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT48000_ICR": "run-iaorx9o7-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Toxicity_Toxigen_T5_Insert": "run-14xfrjfg-toxigen_aug_insert_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_Toxigen_T5_Substitute": "run-14xfrjfg-toxigen_aug_substitute_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_Toxigen_T5_Translate": "run-14xfrjfg-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_Toxigen_T5_Paraphrase": "run-14xfrjfg-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_Toxigen_T5_ICR": "run-14xfrjfg-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",

    # Falcon
    "BOSS_Toxicity_Toxigen_Falcon_Insert": "run-14xfrjfg-toxigen_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_Toxigen_Falcon_Substitute": "run-14xfrjfg-toxigen_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_Toxigen_Falcon_Translate": "run-14xfrjfg-toxigen_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_Toxigen_Falcon_Paraphrase": "run-14xfrjfg-toxigen_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_Toxigen_Falcon_ICR": "run-14xfrjfg-toxigen_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # ImplicitHate
    # BERT
    "BOSS_Toxicity_ImplicitHate_BERT_Insert": "run-57a9z4m8-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_BERT_Substitute": "run-57a9z4m8-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_BERT_Translate": "run-57a9z4m8-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_BERT_Paraphrase": "run-57a9z4m8-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_BERT_ICR": "run-lixlm6qh-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_Insert": "run-ms2943fk-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_Substitute": "run-ms2943fk-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_Translate": "run-ms2943fk-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_Paraphrase": "run-ms2943fk-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_ICR": "run-ms2943fk-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT6000_Insert": "run-2ecsgmy9-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT6000_Substitute": "run-2ecsgmy9-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT6000_Translate": "run-2ecsgmy9-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT6000_Paraphrase": "run-2ecsgmy9-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT6000_ICR": "run-2ecsgmy9-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT12000_Insert": "run-7hgtyr98-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT12000_Substitute": "run-7hgtyr98-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT12000_Translate": "run-7hgtyr98-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT12000_Paraphrase": "run-7hgtyr98-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT12000_ICR": "run-7hgtyr98-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT24000_Insert": "run-wl2a38kf-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT24000_Substitute": "run-wl2a38kf-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT24000_Translate": "run-wl2a38kf-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT24000_Paraphrase": "run-wl2a38kf-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT24000_ICR": "run-wl2a38kf-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT48000_Insert": "run-iaorx9o7-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT48000_Substitute": "run-iaorx9o7-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT48000_Translate": "run-iaorx9o7-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT48000_Paraphrase": "run-iaorx9o7-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT48000_ICR": "run-iaorx9o7-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Toxicity_ImplicitHate_T5_Insert": "run-14xfrjfg-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_T5_Substitute": "run-14xfrjfg-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_T5_Translate": "run-14xfrjfg-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_T5_Paraphrase": "run-14xfrjfg-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_T5_ICR": "run-14xfrjfg-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",

    # Falcon
    "BOSS_Toxicity_ImplicitHate_Falcon_Insert": "run-7nqc8p48-implicit_hate_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_Falcon_Substitute": "run-mzhz5px0-implicit_hate_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_Falcon_Translate": "run-gx7ucgnh-implicit_hate_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_Falcon_Paraphrase": "run-lock4bx1-implicit_hate_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_Falcon_ICR": "run-tr8mluz2-implicit_hate_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # AdvCivil
    # BERT
    "BOSS_Toxicity_AdvCivil_BERT_Insert": "run-57a9z4m8-adv_civil_aug_insert_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_BERT_Substitute": "run-57a9z4m8-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_BERT_Translate": "run-57a9z4m8-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_BERT_Paraphrase": "run-57a9z4m8-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_BERT_ICR": "run-57a9z4m8-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT3000_Insert": "run-ms2943fk-adv_civil_aug_insert_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT3000_Substitute": "run-ms2943fk-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT3000_Translate": "run-ms2943fk-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT3000_Paraphrase": "run-ms2943fk-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT3000_ICR": "run-ms2943fk-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT6000_Insert": "run-2ecsgmy9-adv_civil_aug_insert_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT6000_Substitute": "run-2ecsgmy9-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT6000_Translate": "run-2ecsgmy9-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT6000_Paraphrase": "run-2ecsgmy9-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT6000_ICR": "run-2ecsgmy9-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT12000_Insert": "run-7hgtyr98-adv_civil_aug_insert_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT12000_Substitute": "run-7hgtyr98-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT12000_Translate": "run-7hgtyr98-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT12000_Paraphrase": "run-7hgtyr98-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT12000_ICR": "run-7hgtyr98-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT24000_Insert": "run-wl2a38kf-adv_civil_aug_insert_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT24000_Substitute": "run-wl2a38kf-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT24000_Translate": "run-wl2a38kf-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT24000_Paraphrase": "run-wl2a38kf-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT24000_ICR": "run-wl2a38kf-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT48000_Insert": "run-iaorx9o7-adv_civil_aug_insert_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT48000_Substitute": "run-iaorx9o7-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT48000_Translate": "run-iaorx9o7-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT48000_Paraphrase": "run-iaorx9o7-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT48000_ICR": "run-iaorx9o7-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Toxicity_AdvCivil_T5_Insert": "run-14xfrjfg-adv_civil_aug_insert_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_T5_Substitute": "run-14xfrjfg-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_T5_Translate": "run-14xfrjfg-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_T5_Paraphrase": "run-14xfrjfg-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_T5_ICR": "run-14xfrjfg-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",

    # Falcon
    "BOSS_Toxicity_AdvCivil_Falcon_Insert": "run-14xfrjfg-adv_civil_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_Falcon_Substitute": "run-14xfrjfg-adv_civil_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_Falcon_Translate": "run-14xfrjfg-adv_civil_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_Falcon_Paraphrase": "run-14xfrjfg-adv_civil_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_Falcon_ICR": "run-14xfrjfg-adv_civil_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # AG News Tweets
    # ID
    # BERT
    "AgNewsTweets_ID_BERT_Insert": "run-r4cog6cc-validation_aug_insert_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_ID_BERT_Substitute": "run-r4cog6cc-validation_aug_substitute_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_ID_BERT_Translate": "run-r4cog6cc-validation_aug_backtranslate_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_ID_BERT_Paraphrase": "run-r4cog6cc-validation_stabilityaiStableBeluga7b_random_0_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_ID_BERT_ICR": "run-r4cog6cc-validation_stabilityaiStableBeluga7b_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Insert": "run-s51a9946-test_aug_insert_random_16_Kyle1668agnews4800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Substitute": "run-s51a9946-test_aug_substitute_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Translate": "run-s51a9946-test_aug_backtranslate_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Paraphrase": "run-s51a9946-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_ICR": "run-s51a9946-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Insert": "run-ymlu6tfz-test_aug_insert_random_16_Kyle1668agnews4800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Substitute": "run-ymlu6tfz-test_aug_substitute_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Translate": "run-ymlu6tfz-test_aug_backtranslate_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Paraphrase": "run-ymlu6tfz-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_ICR": "run-ymlu6tfz-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Insert": "run-arskmac6-test_aug_insert_random_16_Kyle1668agnews4800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Substitute": "run-arskmac6-test_aug_substitute_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Translate": "run-arskmac6-test_aug_backtranslate_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Paraphrase": "run-arskmac6-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_ICR": "run-arskmac6-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Insert": "run-6qktrbp8-test_aug_insert_random_16_Kyle1668agnews4800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Substitute": "run-6qktrbp8-test_aug_substitute_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Translate": "run-6qktrbp8-test_aug_backtranslate_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Paraphrase": "run-6qktrbp8-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_ICR": "run-6qktrbp8-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Insert": "run-seqenj7d-test_aug_insert_random_16_Kyle1668agnews4800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Substitute": "run-seqenj7d-test_aug_substitute_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Translate": "run-seqenj7d-test_aug_backtranslate_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Paraphrase": "run-seqenj7d-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_ICR": "run-seqenj7d-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",

    # T5
    "AgNewsTweets_ID_T5_Insert": "run-r4cog6cc-validation_aug_insert_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_ID_T5_Substitute": "run-r4cog6cc-validation_aug_substitute_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_ID_T5_Translate": "run-r4cog6cc-validation_aug_backtranslate_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_ID_T5_Paraphrase": "run-r4cog6cc-validation_stabilityaiStableBeluga7b_random_0_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_ID_T5_ICR": "run-r4cog6cc-validation_stabilityaiStableBeluga7b_random_16_Kyle1668agnewst5large_style_logs:v0",

    # Falcon
    "AgNewsTweets_ID_Falcon_Insert": "run-r4cog6cc-validation_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_ID_Falcon_Substitute": "run-r4cog6cc-validation_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_ID_Falcon_Translate": "run-r4cog6cc-validation_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_ID_Falcon_Paraphrase": "run-r4cog6cc-validation_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_ID_Falcon_ICR": "run-r4cog6cc-validation_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # Tweets
    # BERT
    "AgNewsTweets_Tweets_BERT_Insert": "run-r4cog6cc-test_aug_insert_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Tweets_BERT_Substitute": "run-r4cog6cc-test_aug_substitute_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Tweets_BERT_Translate": "run-r4cog6cc-test_aug_backtranslate_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Tweets_BERT_Paraphrase": "run-r4cog6cc-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Tweets_BERT_ICR": "run-r4cog6cc-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Insert": "run-seqenj7d-test_aug_insert_random_16_Kyle1668agnews76800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Substitute": "run-seqenj7d-test_aug_substitute_random_16_Kyle1668agnews76800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Translate": "run-seqenj7d-test_aug_backtranslate_random_16_Kyle1668agnews76800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Paraphrase": "run-seqenj7d-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnews76800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_ICR": "run-seqenj7d-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnews76800bertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Insert": "run-ymlu6tfz-test_aug_insert_random_16_Kyle1668agnews9600bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Substitute": "run-ymlu6tfz-test_aug_substitute_random_16_Kyle1668agnews9600bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Translate": "run-ymlu6tfz-test_aug_backtranslate_random_16_Kyle1668agnews9600bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Paraphrase": "run-ymlu6tfz-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnews9600bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_ICR": "run-ymlu6tfz-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnews9600bertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Insert": "run-arskmac6-test_aug_insert_random_16_Kyle1668agnews19200bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Substitute": "run-arskmac6-test_aug_substitute_random_16_Kyle1668agnews19200bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Translate": "run-arskmac6-test_aug_backtranslate_random_16_Kyle1668agnews19200bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Paraphrase": "run-arskmac6-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnews19200bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_ICR": "run-arskmac6-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnews19200bertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Insert": "run-6qktrbp8-test_aug_insert_random_16_Kyle1668agnews38400bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Substitute": "run-6qktrbp8-test_aug_substitute_random_16_Kyle1668agnews38400bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Translate": "run-6qktrbp8-test_aug_backtranslate_random_16_Kyle1668agnews38400bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Paraphrase": "run-6qktrbp8-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnews38400bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_ICR": "run-6qktrbp8-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnews38400bertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Insert": "run-s51a9946-test_aug_insert_random_16_Kyle1668agnews4800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Substitute": "run-s51a9946-test_aug_substitute_random_16_Kyle1668agnews4800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Translate": "run-s51a9946-test_aug_backtranslate_random_16_Kyle1668agnews4800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Paraphrase": "run-s51a9946-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnews4800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_ICR": "run-s51a9946-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnews4800bertbaseuncased_style_logs:v0",

    # T5
    "AgNewsTweets_Tweets_T5_Insert": "run-r4cog6cc-test_aug_insert_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_Tweets_T5_Substitute": "run-r4cog6cc-test_aug_substitute_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_Tweets_T5_Translate": "run-r4cog6cc-test_aug_backtranslate_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_Tweets_T5_Paraphrase": "run-r4cog6cc-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_Tweets_T5_ICR": "run-r4cog6cc-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnewst5large_style_logs:v0",

    # Falcon
    "AgNewsTweets_Tweets_Falcon_Insert": "run-r4cog6cc-test_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_Tweets_Falcon_Substitute": "run-r4cog6cc-test_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_Tweets_Falcon_Translate": "run-r4cog6cc-test_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_Tweets_Falcon_Paraphrase": "run-r4cog6cc-test_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_Tweets_Falcon_ICR": "run-r4cog6cc-test_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",
}

print(json.dumps(table_paths, indent=4))

{
    "BOSS_Sentiment_ID_BERT_Insert": "run-omxsb0i7-validation_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_BERT_Substitute": "run-omxsb0i7-validation_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_BERT_Translate": "run-omxsb0i7-validation_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_BERT_Paraphrase": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_BERT_ICR": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Insert": "run-omxsb0i7-validation_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Substitute": "run-omxsb0i7-validation_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_

In [3]:
run = wandb.init()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kyledevinobrien1 (healthyml). Use `wandb login --relogin` to force relogin


In [4]:
def get_local_logs_path(directory):
    try:
        for contents in os.listdir(directory):
            if contents.endswith("style_logs.table.json"):
                return os.path.join(directory, contents)

            if os.path.isdir(os.path.join(directory, contents)):
                return get_local_logs_path(os.path.join(directory, contents))
    except:
        # print("No style logs found in {}".format(directory))
        return None

print(get_local_logs_path("data/BOSS_Toxicity_ID_Falcon_Paraphrase"))

data/BOSS_Toxicity_ID_Falcon_Paraphrase/validation_stabilityai/StableBeluga-7b_random_0_tiiuae/falcon-7b-instruct_style_logs.table.json


In [5]:
# for each table path, run.use_artifact and download the artifact.
local_paths = {}
missing_tables = []
for table_name, table_path in tqdm(list(table_paths.items())[:]):
    # if table_name == "BOSS_Toxicity_ImplicitHate_BERT_ICR":
    #     print()
    # else:
    #     continue

    try:
        if table_path == "":
            missing_tables.append(table_name)
            continue

        if get_local_logs_path(f"data/{table_name}") is None:
            download_path = f"{project_name}/{table_path}"
            print(f"Downloading {table_name} --- {download_path}")
            artifact = run.use_artifact(download_path, type="run_table")
            artifact_dir = artifact.download(root=f"data/{table_name}")

        local_paths[table_name] = get_local_logs_path(f"data/{table_name}")
    except Exception as e:
        print(e)
        print(table_name)
        missing_tables.append(table_name)

print(json.dumps(missing_tables, indent=4))

  0%|          | 0/325 [00:00<?, ?it/s]

wandb: Downloading large artifact run-mzhz5px0-implicit_hate_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0, 658.58MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:2:45.2
 70%|██████▉   | 227/325 [02:45<01:11,  1.37it/s]

wandb: Downloading large artifact run-gx7ucgnh-implicit_hate_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0, 658.12MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:2:52.0
 70%|███████   | 228/325 [05:38<02:55,  1.81s/it]

wandb: Downloading large artifact run-lock4bx1-implicit_hate_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0, 665.47MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:2:47.4
 70%|███████   | 229/325 [08:27<05:15,  3.28s/it]

wandb: Downloading large artifact run-tr8mluz2-implicit_hate_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0, 710.62MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:4:47.7
100%|██████████| 325/325 [13:16<00:00,  2.45s/it]

[]


In [6]:
column_renaming = {
    "entropy": "tta_prediction_entropy",
    "mean probs": "tta_mean_class_probs",
    "all probs": "tta_all_class_probs",
    "all entropies": "tta_all_class_entropy",
    "latency": "tta_inference_latency",
    "input": "augmentations",
    "original_input": "original_text",
    "judgment": "tta_predicted_class",
    "original judgment": "original_predicted_class",
    "original entropy": "original_prediction_entropy",
    "entropy decrease": "prediction_entropy_decrease",
    "entropy decreased": "prediction_entropy_decreased",
}
columns_to_drop = ["style prompt", "mean exemplar distance", "prompt"]
columns_order = ["outcome", "original_text", "augmentations", "generations", "original_predicted_class", "tta_predicted_class", "label", "tta_inference_latency", "original_prediction_entropy", "tta_prediction_entropy", "prediction_entropy_decreased", "prediction_entropy_decrease", "tta_mean_class_probs", "tta_all_class_probs", "tta_all_class_entropy", ]

In [7]:
def parse_augmentations(augmentations):
    if augmentations is None or len(augmentations) == 0:
        return []

    return [aug.replace("<aug>", "").replace("</aug>", "") for aug in augmentations[:4]]

formatted_augs = parse_augmentations([ "<aug>too not much of anything this well - well acted but dangerously slow thriller feels somehow like simply a terrible preamble to a bigger, much more too complicated horror story, one reality that never materializes.</aug>", "<aug>one too too much apprehension of this well - being acted but dangerously still slow thriller feels like only a preamble to a seemingly bigger, yet more more complicated story, and one that never materializes.</aug>", "<aug>too much of this a well - well acted but dangerously slow thriller film feels like a big preamble opening to a somehow bigger, more complicated dream story, the one concept that almost never materializes.</aug>", "<aug>too am much of this this well - acted but dangerously slow paced thriller feels somehow like a prime preamble to creating a bigger, more painfully complicated story, but one one that never ever materializes.</aug>", "<aug>Too much of this well-acted but dangerously slow thriller feels like a preamble to a bigger , more complicated story , one that never materializes .</aug>", "<aug>Too much of this well-acted but dangerously slow thriller feels like a preamble to a bigger , more complicated story , one that never materializes .</aug>" ])

print(formatted_augs)
print(len(formatted_augs))

['too not much of anything this well - well acted but dangerously slow thriller feels somehow like simply a terrible preamble to a bigger, much more too complicated horror story, one reality that never materializes.', 'one too too much apprehension of this well - being acted but dangerously still slow thriller feels like only a preamble to a seemingly bigger, yet more more complicated story, and one that never materializes.', 'too much of this a well - well acted but dangerously slow thriller film feels like a big preamble opening to a somehow bigger, more complicated dream story, the one concept that almost never materializes.', 'too am much of this this well - acted but dangerously slow paced thriller feels somehow like a prime preamble to creating a bigger, more painfully complicated story, but one one that never ever materializes.']
4


In [8]:
local_paths

{'BOSS_Sentiment_ID_BERT_Insert': 'data/BOSS_Sentiment_ID_BERT_Insert/validation_aug_insert_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json',
 'BOSS_Sentiment_ID_BERT_Substitute': 'data/BOSS_Sentiment_ID_BERT_Substitute/validation_aug_substitute_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json',
 'BOSS_Sentiment_ID_BERT_Translate': 'data/BOSS_Sentiment_ID_BERT_Translate/validation_aug_back-translate_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json',
 'BOSS_Sentiment_ID_BERT_Paraphrase': 'data/BOSS_Sentiment_ID_BERT_Paraphrase/validation_stabilityai/StableBeluga-7b_random_0_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json',
 'BOSS_Sentiment_ID_BERT_ICR': 'data/BOSS_Sentiment_ID_BERT_ICR/validation_stabilityai/StableBeluga-7b_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json',
 'BOSS_Sentiment_ID_T5_Insert': 'data/BOSS_Sentiment_ID_T5_Insert/validation_aug_insert_random_16_K

In [9]:
combined_datset = DatasetDict()
for split in tqdm(local_paths):
    # print(f"Processing {split}")
    path = local_paths[split]
    json_logs = json.load(open(path))
    frame = pd.DataFrame(data=json_logs["data"], columns=json_logs["columns"])
    frame = frame.rename(columns=column_renaming)
    frame = frame.drop(columns=columns_to_drop)
    frame["augmentations"] = frame["augmentations"].apply(parse_augmentations)

    if "BERT" not in split:
        for col in columns_order:
            if col not in frame.columns:
                if col == "original_prediction_entropy" or col == "tta_prediction_entropy" or col == "prediction_entropy_decrease":
                    frame[col] = float("nan")
                elif col == "prediction_entropy_decreased":
                    frame[col] = False
                elif col == "tta_all_class_probs":
                    frame[col] = frame.apply(lambda x: [[float(0.0)]], axis=1)
                elif "probs" in col:
                    frame[col] = frame.apply(lambda x: [float(0.0)], axis=1)
                elif col == "tta_all_class_entropy":
                    frame[col] = frame.apply(lambda x: [float(0.0)], axis=1)

    if "generations" not in frame.columns:
        frame["generations"] = [[]] * len(frame)

    frame = frame[columns_order]
    combined_datset[split] = Dataset.from_pandas(frame)

combined_datset

100%|██████████| 325/325 [02:01<00:00,  2.68it/s]


DatasetDict({
    BOSS_Sentiment_ID_BERT_Insert: Dataset({
        features: ['outcome', 'original_text', 'augmentations', 'generations', 'original_predicted_class', 'tta_predicted_class', 'label', 'tta_inference_latency', 'original_prediction_entropy', 'tta_prediction_entropy', 'prediction_entropy_decreased', 'prediction_entropy_decrease', 'tta_mean_class_probs', 'tta_all_class_probs', 'tta_all_class_entropy'],
        num_rows: 38904
    })
    BOSS_Sentiment_ID_BERT_Substitute: Dataset({
        features: ['outcome', 'original_text', 'augmentations', 'generations', 'original_predicted_class', 'tta_predicted_class', 'label', 'tta_inference_latency', 'original_prediction_entropy', 'tta_prediction_entropy', 'prediction_entropy_decreased', 'prediction_entropy_decrease', 'tta_mean_class_probs', 'tta_all_class_probs', 'tta_all_class_entropy'],
        num_rows: 38904
    })
    BOSS_Sentiment_ID_BERT_Translate: Dataset({
        features: ['outcome', 'original_text', 'augmentations', 'gen

In [10]:
combined_datset.save_to_disk("data/combined_dataset")

Saving the dataset (1/1 shards): 100%|██████████| 7600/7600 [00:00<00:00, 1360566.41 examples/s]


In [11]:
list(combined_datset.keys())

['BOSS_Sentiment_ID_BERT_Insert',
 'BOSS_Sentiment_ID_BERT_Substitute',
 'BOSS_Sentiment_ID_BERT_Translate',
 'BOSS_Sentiment_ID_BERT_Paraphrase',
 'BOSS_Sentiment_ID_BERT_ICR',
 'BOSS_Sentiment_ID_T5_Insert',
 'BOSS_Sentiment_ID_T5_Substitute',
 'BOSS_Sentiment_ID_T5_Translate',
 'BOSS_Sentiment_ID_T5_Paraphrase',
 'BOSS_Sentiment_ID_T5_ICR',
 'BOSS_Sentiment_ID_Falcon_Insert',
 'BOSS_Sentiment_ID_Falcon_Substitute',
 'BOSS_Sentiment_ID_Falcon_Translate',
 'BOSS_Sentiment_ID_Falcon_Paraphrase',
 'BOSS_Sentiment_ID_Falcon_ICR',
 'BOSS_Sentiment_SST5_BERT_Insert',
 'BOSS_Sentiment_SST5_BERT_Substitute',
 'BOSS_Sentiment_SST5_BERT_Translate',
 'BOSS_Sentiment_SST5_BERT_Paraphrase',
 'BOSS_Sentiment_SST5_BERT_ICR',
 'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Insert',
 'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Substitute',
 'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Translate',
 'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Paraphrase',
 'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_ICR',
 'BOSS_Sen

In [12]:
# combined_datset.push_to_hub("LLM-TTA-Augmentation-Logs")